In [21]:
import pandas as pd
import json

import requests

import time

import os, codecs

In [79]:
# Establishing access to Twitter API
with open('keychain.json') as f:
  keychain = json.load(f)

# Using Twython for rehybdrating individual tweets
twitter = Twython(keychain['CONSUMER_KEY'], keychain['CONSUMER_SECRET'], oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(keychain['CONSUMER_KEY'], access_token=ACCESS_TOKEN)

First, we need to gain access to a list of tweets that we would like to resurrect. 

We have previously shared a list of tweet ids related to hello brother and provide a Jupyter notebook for rehydrating the tweets. 

Next, we will continue the process after the tweets have been rehydrated.

In [23]:
# Reading the JSON Lines data
with open('data/hellobrother_rehydrated.jsonl') as f:
    tweets_jsonl = f.read()

# Creating a Pandas dataframe    
df_hydrated = pd.DataFrame([json.loads(jline) for jline in tweets_jsonl.splitlines()]).sort_values('retweet_count', ascending=False)

# Selecting the tweets with most retweets
df_hydrated.head(5).full_text

1455     "Daoud Nabi", a 71-year-old Afghan refugee who...
21677    Selam kardeşim...\n\n#HelloBrother https://t.c...
4206     Almighty, we pray for those grieving. Those wh...
24336    Terrorism has no religion. Let us unite and st...
1131     ظلموا شريعة خير من وطئ الثرى\nقالوا: عُطاشى لل...
Name: full_text, dtype: object

In [11]:
df_hydrated.columns

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'source', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo',
       'coordinates', 'place', 'contributors', 'is_quote_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'retweet_count', 'favorite_count', 'favorited', 'retweeted',
       'possibly_sensitive', 'lang', 'quoted_status', 'extended_entities',
       'withheld_in_countries'],
      dtype='object')

Next, extracting author username for each tweet for easy accessibility

In [24]:
def extract_username(user):
    #print(user)
    return user['screen_name']

df_hydrated['author_username'] = df_hydrated.user.apply(extract_username)

Using Twitter's oEmbed application programming interface (API) to fetch the necessary data on representing tweets in hypertext markup language (HTML).

In [27]:
embeds = list()

for tweet in df_hydrated.head(10).itertuples():
    print(tweet)
    # print(tweet.author_username)
    tweet_url = 'https://twitter.com/%s/status/%s' % (tweet.author_username, tweet.id)
    print(tweet_url)

    response = requests.get('https://publish.twitter.com/oembed?url={}'.format(tweet_url))
    
    # embed = twitter.get_oembed_tweet(url=tweet_url)

    # Checking whether we can still access the tweet
    if response.status_code == 200:
        embeds.append(response.json())

    time.sleep(2)






Pandas(Index=1455, created_at='Fri Mar 15 19:57:42 +0000 2019', id=1106645666472886272, id_str='1106645666472886272', full_text='"Daoud Nabi", a 71-year-old Afghan refugee who escaped death in his country to cite in New Zealand where he received the terrorist at the mosque door with the words "welcome brother" but the terrorist killed him directly\n#HelloBrother #NewZealandTerroristAttack https://t.co/BwMKujJXjp', truncated=False, display_text_range=[0, 261], entities={'hashtags': [{'text': 'HelloBrother', 'indices': [221, 234]}, {'text': 'NewZealandTerroristAttack', 'indices': [235, 261]}], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1106645662400163844, 'id_str': '1106645662400163844', 'indices': [262, 285], 'media_url': 'http://pbs.twimg.com/media/D1uYXd-WsAQHJY7.jpg', 'media_url_https': 'https://pbs.twimg.com/media/D1uYXd-WsAQHJY7.jpg', 'url': 'https://t.co/BwMKujJXjp', 'display_url': 'pic.twitter.com/BwMKujJXjp', 'expanded_url': 'https://twitter.com/FahadAldaw

Creating a new dataframe for the embeddings

In [28]:
df_embeds = pd.DataFrame(embeds)
df_embeds

,url,author_name,author_url,html,width,height,type,cache_age,provider_name,provider_url,version
0,https://twitter.com/Fahad_Alammar8/status/1106...,م. فهد العمّار Fahad Alammar,https://twitter.com/Fahad_Alammar8,"<blockquote class=""twitter-tweet""><p lang=""en""...",550,None,rich,3153600000,Twitter,https://twitter.com,1.0
1,https://twitter.com/RTErdogan/status/110938900...,Recep Tayyip Erdoğan,https://twitter.com/RTErdogan,"<blockquote class=""twitter-tweet""><p lang=""tr""...",550,None,rich,3153600000,Twitter,https://twitter.com,1.0
2,https://twitter.com/muftimenk/status/110666897...,Mufti Menk,https://twitter.com/muftimenk,"<blockquote class=""twitter-tweet""><p lang=""en""...",550,None,rich,3153600000,Twitter,https://twitter.com,1.0
3,https://twitter.com/M10/status/111017020574818...,Mesut Özil,https://twitter.com/M10,"<blockquote class=""twitter-tweet""><p lang=""en""...",550,None,rich,3153600000,Twitter,https://twitter.com,1.0
4,https://twitter.com/Dr_alqarnee/status/1106633...,د. عائض القرني,https://twitter.com/Dr_alqarnee,"<blockquote class=""twitter-tweet""><p lang=""ar""...",550,None,rich,3153600000,Twitter,https://twitter.com,1.0
5,https://twitter.com/Dr_shathry/status/11066750...,د. علي الشثري,https://twitter.com/Dr_shathry,"<blockquote class=""twitter-tweet""><p lang=""ar""...",550,None,rich,3153600000,Twitter,https://twitter.com,1.0
6,https://twitter.com/AbdullahK5/status/11066638...,عـبدالله الخريّف,https://twitter.com/AbdullahK5,"<blockquote class=""twitter-tweet""><p lang=""und...",550,None,rich,3153600000,Twitter,https://twitter.com,1.0
7,https://twitter.com/rightZaid/status/110651860...,زَيْد,https://twitter.com/rightZaid,"<blockquote class=""twitter-tweet""><p lang=""ar""...",550,None,rich,3153600000,Twitter,https://twitter.com,1.0
8,https://twitter.com/emtinan4444/status/1106700...,امــتـنـان,https://twitter.com/emtinan4444,"<blockquote class=""twitter-tweet""><p lang=""en""...",550,None,rich,3153600000,Twitter,https://twitter.com,1.0


Creating the resurrection view by constructing a simple page in hypertext markup language (HTML). One can use more sophisticated means to generate the page, including [Jinja](https://pypi.org/project/Jinja2/).

In [29]:
prefix = """<style type="text/css">
    .tweet_container {
      float: left;
      display: inline;
      min-height: 500px;
      max-width: 550px;
      padding: 10px;
    }
    
    .twitter-tweet {
    }
  </style>
  """

suffix = ''


In [88]:
from IPython.core.display import HTML

In [31]:
'''
Function for generating the representation of a single tweet
'''
def render_tweets(df, html):
    for embed in df_embeds.itertuples():
        html.write('<div class="tweet_container">')
        html.write(embed.html)
        html.write('</div>')

In [90]:
if not os.path.exists('resurrected'):
    # Create the folder
    os.makedirs('resurrected')

with codecs.open('resurrected/tweets-sample.html', 'w', encoding='utf-8') as html:
  html.write(prefix)
  render_tweets(df_hydrated, html)
  html.write(suffix)

